In [40]:
import numpy as np
import tensorflow as tf
# import matplotlib.pyplot as plt

from PIL import Image

# ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.models import Model

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from keras.optimizers import Adam
from keras import backend as K

# from sklearn.metrics import confusion_matrix

In [41]:
physical_devices = tf.config.list_physical_devices('GPU')
for i, device in enumerate(physical_devices):
    print(f"GPU {i}: {device}")


GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
GPU 1: PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [42]:
path = 'car_make_images/'
training_path = path + 'train'
testing_path = path + 'test'
validation_path = path + 'val'

In [43]:
# Inicializamos los ImageDataGenerator

training_data_generator = ImageDataGenerator(rescale = 1./255,
                              rotation_range = 359,
                              shear_range = 0.2,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              zoom_range = 0.2,
                              horizontal_flip = True,
                              vertical_flip = True,
                              preprocessing_function = None)

validation_data_generator = ImageDataGenerator(rescale = 1./255)  # Removed augmentation
test_data_generator = ImageDataGenerator(rescale = 1./255)  # Removed augmentation

In [44]:
# Entrenamos los ImageDataGenerator

# Para poder usar .flow_from_directory(), necesitamos tener las carpetas ordenadas, es decir:

# una carpeta de train con una carpeta de imagenes para cada clase
# una carpeta de validation con una carpeta de imagenes para cada clase
# y una carpeta de test con una carpeta de imagenes para cada clase

size = 200
batch_size = 30  # For example
num_classes = 39  # Change to the number of classes you actually have

training_generator = training_data_generator.flow_from_directory(training_path,
                                                                 target_size = (size, size),
                                                                 batch_size = 30,
                                                                 class_mode = "categorical")

validation_generator = validation_data_generator.flow_from_directory(validation_path,
                                                                     target_size = (size, size),
                                                                     batch_size = 1,
                                                                     class_mode = "categorical")

test_generator = test_data_generator.flow_from_directory(testing_path,
                                                         target_size = (size, size),
                                                         batch_size = 1,
                                                         class_mode = "categorical")

# En caso de que la clasificación sea multi-clase, el cambiamos el parámetro class_mode = "categorical"

Found 11573 images belonging to 39 classes.
Found 2813 images belonging to 39 classes.
Found 2871 images belonging to 39 classes.


In [45]:
# with tf.device('/GPU:0'):

#     model = Sequential()

#     model.add(Input(shape = (size, size, 3)))

#     model.add(Conv2D(filters = 16, kernel_size = 9, padding = "same", activation = "relu"))
#     model.add(MaxPooling2D(pool_size = 2))

#     model.add(Conv2D(filters = 16, kernel_size = 9, padding = "same", activation = "relu"))
#     model.add(MaxPooling2D(pool_size = 2))

#     model.add(Flatten())

#     model.add(Dense(units = 120, activation = "relu"))
#     model.add(Dense(units = 120, activation = "relu"))
#     model.add(Dense(units = 120, activation = "relu"))
#     model.add(Dropout(0.15))
#     model.add(Dense(units = num_classes, activation = "softmax"))  # Changed to 'softmax'
    
#     # Model Compilation
#     model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

#     # Model Training
#     history = model.fit(training_generator, epochs = 1, validation_data = validation_generator)

In [46]:
def f2_score(y_true, y_pred):
    beta_squared = 4  # beta=2, beta^2 = 4
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    fp = K.sum(K.round(K.clip(y_pred - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f2_val = (1 + beta_squared) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())
    return f2_val

In [48]:
def create_model(size, num_classes):
    model = Sequential()

    model.add(Input(shape=(size, size, 3)))

    # First Conv Block
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(MaxPooling2D(pool_size=2))

    # Second Conv Block
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(MaxPooling2D(pool_size=2))

    # Third Conv Block
    model.add(Conv2D(filters=128, kernel_size=3, padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(MaxPooling2D(pool_size=2))

    # Fourth Conv Block
    model.add(Conv2D(filters=256, kernel_size=3, padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(tf.keras.layers.ReLU())
    model.add(MaxPooling2D(pool_size=2))

    # Flatten and Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.15))
    model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(num_classes, activation='softmax', kernel_initializer='he_normal'))

    return model


In [49]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import os

def train_models(learning_rates, batch_sizes, training_generator, validation_generator):
    results = []
    
    for lr in learning_rates:
        for batch in batch_sizes:
            
            # Define the learning rate schedule
            lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=lr,
                decay_steps=10000,
                decay_rate=0.9)
            opt = Adam(learning_rate=lr_schedule)
            
            # Define early stopping
            early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
            
            # Define the model (assumes you have a function that returns your model)
            model = create_model(size, num_classes)  # Replace this with your actual model function
            model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
            
            # Train the model
            history = model.fit(training_generator, epochs=100, batch_size=batch,
                                validation_data=validation_generator, 
                                callbacks=[early_stopping])
            
            # Save the model
            model_name = f'model_lr_{lr}_batch_{batch}.h5'
            model.save(model_name)
            
            # Save metrics
            metrics_df = pd.DataFrame(history.history)
            metrics_name = f'metrics_lr_{lr}_batch_{batch}.csv'
            metrics_df.to_csv(metrics_name, index=False)
            
            # Save the number of epochs trained and append to results
            n_epochs = len(history.history['accuracy'])
            results.append({
                'learning_rate': lr,
                'batch_size': batch,
                'epochs_trained': n_epochs,
                'final_val_accuracy': history.history['val_accuracy'][-1]
            })

    # Save results to a CSV file
    results_df = pd.DataFrame(results)
    results_df.to_csv('training_results.csv', index=False)

In [50]:
# Example usage:
learning_rates = [0.005, 0.01] # [0.001, 0.005, 0.01]
batch_sizes = [32, 64]
with tf.device("/GPU:0"):
    train_models(learning_rates, batch_sizes, training_generator, validation_generator)

Epoch 1/100
386/386 [==============================] - 137s 352ms/step - loss: 5.4602 - accuracy: 0.0356 - val_loss: 3.6256 - val_accuracy: 0.0398
Epoch 2/100
386/386 [==============================] - 181s 469ms/step - loss: 3.6295 - accuracy: 0.0444 - val_loss: 3.6210 - val_accuracy: 0.0462
Epoch 3/100
386/386 [==============================] - 149s 385ms/step - loss: 3.6213 - accuracy: 0.0440 - val_loss: 3.6191 - val_accuracy: 0.0462
Epoch 4/100
386/386 [==============================] - 166s 431ms/step - loss: 3.6210 - accuracy: 0.0442 - val_loss: 3.6189 - val_accuracy: 0.0462
Epoch 5/100
386/386 [==============================] - 149s 384ms/step - loss: 3.6208 - accuracy: 0.0442 - val_loss: 3.6188 - val_accuracy: 0.0462
Epoch 6/100
386/386 [==============================] - 140s 364ms/step - loss: 3.6210 - accuracy: 0.0442 - val_loss: 3.6189 - val_accuracy: 0.0462
Epoch 7/100
386/386 [==============================] - 148s 383ms/step - loss: 3.6209 - accuracy: 0.0440 - val_loss: 3

KeyboardInterrupt: 